# Punti di interesse

# Municipi

In [2]:
import fiona
import fiona.transform
import json

In [ ]:
PATH_MUNICIPI_RAW = "../Data/Raw/Punti_di_interesse/Municipi/Municipi.shp"
PATH_MUNICIPI_JSON = "../Data/Clean/Punti_di_interesse/municipi_clean.geojson"

Carichiamo il file .shp (shape file) e scriviamolo in Clean in formato geojson

In [ ]:
src_crs = 'EPSG:6707'
dst_crs = 'EPSG:4326'

with fiona.open(PATH_MUNICIPI_RAW, 'r') as src:
    with fiona.open(PATH_MUNICIPI_JSON, 'w', driver='GeoJSON', crs=src.crs, schema=src.schema) as dst:
        for feature in src:
            feature['geometry'] = fiona.transform.transform_geom(
                src_crs,
                dst_crs,
                feature['geometry']
            )
            dst.write(feature)

# Solo per una corretta indentazione
with open(PATH_MUNICIPI_JSON, 'r') as file:
    file_json = json.load(file)

with open(PATH_MUNICIPI_JSON, 'w') as file:
    json.dump(file_json, file, indent=4)

# PARCHI

In [4]:
import fiona
import json

In [ ]:
PATH_PARCHI_RAW = "../Data/Raw/Punti_di_interesse/Parchi/Parchi_WGS84_Milano_1.shp"
PATH_PARCHI_CLEAN = "../Data/Clean/Punti_di_interesse/parchi_clean.geojson"

Carichiamo il file .shp (shape file) e scriviamolo in Clean in formato geojson

In [ ]:
with fiona.open(PATH_PARCHI_RAW, 'r') as src:
    with fiona.open(PATH_PARCHI_CLEAN, 'w', driver='GeoJSON', crs=src.crs, schema=src.schema) as dst:
        for feature in src:
            if feature["geometry"]["type"] == "Polygon":
                dst.write(feature)

# Solo per una corretta indentazione
with open(PATH_PARCHI_CLEAN, 'r') as file:
    file_json = json.load(file)
    
with open(PATH_PARCHI_CLEAN, 'w') as file:
    json.dump(file_json, file, indent=4)

# Fontane

In [ ]:
import json

import geopandas as gpd

In [ ]:
PATH_FONTANE_RAW = "../Data/Raw/Punti_di_interesse/fontane_raw.csv"
PATH_FONTANE_CLEAN = "../Data/Clean/Punti_di_interesse/fontane_clean.geojson"

Carichiamo Raw csv

In [ ]:
df = gpd.read_file(PATH_FONTANE_RAW, sep=";")

Ridenominazione colonne

In [13]:
df = df.rename(columns = {'objectID':'id_fontane',
                           'MUNICIPIO':'municipio',
                           'ID_NIL': 'id_nome',
                           'NIL': "nome",
                           'LONG_X_4326':'longitudine',
                           'LAT_Y_4326': 'latitudine',
                           'Location':'location'})

Aggiunta icona per visualizzazione su kepler

In [14]:
df['icon'] = "events"

Conversione da csv a geojson

In [16]:
geojson = {
    "type": "FeatureCollection",
    "name": "",
    "crs": {
        "type": "name",
        "properties": {
            "name": "urn:ogc:def:crs:EPSG::6707"
        }
    },
    "features": []
}
geojson["name"] = "Fontane"
for i in df.index:
    feature = {
        "type": "Feature",
        "properties": {},
        "geometry": {
            "type": "",
            "coordinates": []
            }
        }
    for j in df.columns:
        if j == "location":
            feature["geometry"]["type"] = "Point"
            coordinates = df.at[i, j].strip("(").strip(")").strip().split(",")
            feature["geometry"]["coordinates"].append(float(coordinates[1]))
            feature["geometry"]["coordinates"].append(float(coordinates[0]))
        else:
            feature["properties"][j] = df.at[i, j]
    geojson["features"].append(feature)

Salviamo geojson in Clean

In [ ]:
with open(PATH_FONTANE_CLEAN, 'w') as file:
    json.dump(geojson, file, indent=4)